# A demo of a RAG system with Kotlin notebooks

### Setup api client

In [12]:
@file:DependsOn("io.ktor:ktor-client-core-jvm:2.3.7")
@file:DependsOn("io.ktor:ktor-client-cio-jvm:2.3.7")
@file:DependsOn("io.ktor:ktor-client-content-negotiation-jvm:2.3.7")
@file:DependsOn("io.ktor:ktor-serialization-jackson-jvm:2.3.7")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json-jvm:2.3.7")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.6.3")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core-jvm:1.7.3")

import io.ktor.client.*
import io.ktor.client.call.*
import io.ktor.client.engine.cio.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.client.plugins.contentnegotiation.*
import io.ktor.http.*
import io.ktor.serialization.kotlinx.json.*
import kotlinx.serialization.json.Json
import kotlinx.serialization.*
import kotlinx.serialization.json.*
import kotlinx.serialization.SerialName
import kotlinx.coroutines.runBlocking
import java.util.UUID


In [3]:
@Serializable
data class OpenAiRequest(
    val model: String,
    val messages: List<Message>,
    val temperature: Double = 0.2
)

@Serializable
data class Message(
    @SerialName("role") val role: String = "",
    @SerialName("content") val content: String = ""
)

@Serializable
data class Choice(
    @SerialName("message") val message: Message? = null
)

@Serializable
data class OpenAiResponse(
    @SerialName("choices") val choices: List<Choice> = emptyList()
)

val client = HttpClient(CIO) {
    install(ContentNegotiation) {
        json(Json {
            ignoreUnknownKeys = true  // Avoids issues with fields like "refusal"
        })
    }
}

val apiKey = System.getenv("OPENAI_API_KEY") ?: error("API key not set")



### Read in `.csv`s of products and emails

In [4]:
// Ensures that the latest available library versions are used
%useLatestDescriptors

// Imports the Kotlin DataFrame library
%use dataframe

// Creates a DataFrame by importing data from the "netflix_titles.csv" file.
val emailDataFrame = DataFrame.read("emails.csv")

emailDataFrame

email_id,subject,message
E001,Leather Wallets,"Hi there, I want to order all the rem..."
E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the ..."
E003,Need your help,"Hello, I need a new bag to carry my l..."
E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four S..."
E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl,..."
E006,null,"Hey there, I was thinking of ordering..."
E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF..."
E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your ..."
E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN..."
E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair o..."


In [20]:
data class Classified(val emailId: String, val requestType: String)

val classifiedResults: List<Classified> = emailDataFrame.rows().map { row ->
    val id = row["email_id"] as String
    val subject = row["subject"] as String?
    val message = row["message"] as String

    val promptText = "${subject ?: ""}: $message"

    val request = OpenAiRequest(
        model = "gpt-3.5-turbo",
        messages = listOf(
            Message("system", "Classify this email as 'Product Inquiry' or 'Order Request'. Only return the label."),
            Message("user", promptText)
        )
    )

    val type = runBlocking {
        try {
            client.post("https://api.openai.com/v1/chat/completions") {
                headers {
                    append(HttpHeaders.Authorization, "Bearer $apiKey")
                    append(HttpHeaders.ContentType, ContentType.Application.Json)
                }
                setBody(request)
            }.body<OpenAiResponse>().choices.firstOrNull()?.message?.content?.trim() ?: "Unknown"
        } catch (e: Exception) {
            "Error"
        }
    }

    Classified(id, type)
}

Write out the classifications to a csv file

In [6]:
val classifiedDataFrame = classifiedResults.toDataFrame()

classifiedDataFrame.writeCSV("classified-emails.csv")

classifiedDataFrame

emailId,requestType
E001,Order Request
E002,Product Inquiry
E003,Product Inquiry
E004,Order Request
E005,Product Inquiry
E006,Product Inquiry
E007,Order Request
E008,Order Request
E009,Product Inquiry
E010,Order Request


Read in our csv of products

In [7]:
data class Product(
    val product_id: String,
    val name: String,
    val category: String,
    val description: String,
    val stock: Int,
    val seasons: String,
    val price: Double
)

val productsDataFrame = DataFrame.read("products.csv")
val products = productsDataFrame.toListOf<Product>()

productsDataFrame

product_id,name,category,description,stock,seasons,price
RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with ...,1,"Spring, Summer",26.990000
SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and se...,5,All seasons,30.000000
VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wa...,6,"Spring, Fall",23.000000
CSH1098,Cozy Shawl,Accessories,Wrap yourself in comfort with our coz...,3,"Fall, Winter",22.000000
CHN0987,Chunky Knit Beanie,Accessories,Keep your head toasty with our chunky...,2,"Fall, Winter",22.000000
LTH0976,Leather Bifold Wallet,Accessories,Upgrade your everyday carry with our ...,4,All seasons,21.000000
FZZ1098,Fuzzy Slippers,Accessories,Cozy up in our fuzzy slippers. These ...,2,"Fall, Winter",29.000000
FRP9876,Fringe Poncho,Accessories,Add a boho-chic touch to your outfits...,3,"Fall, Winter",43.000000
BKR0123,Bucket Hat,Accessories,Protect your face from the sun in sty...,3,"Spring, Summer",39.990000
CBY6789,Corduroy Bucket Hat,Accessories,Keep it casual and cool with our cord...,3,"Fall, Winter",28.000000


Setup an embedding request for our data once it's filtered down

In [8]:
@Serializable
data class EmbeddingRequest(val input: String, val model: String)

@Serializable
data class EmbeddingResponse(val data: List<EmbeddingItem>)

@Serializable
data class EmbeddingItem(val embedding: List<Float>)

fun embed(text: String): List<Float> = runBlocking {
    val response = client.post("https://api.openai.com/v1/embeddings") {
        headers {
            append(HttpHeaders.Authorization, "Bearer $apiKey")
            append(HttpHeaders.ContentType, ContentType.Application.Json)
        }
        setBody(EmbeddingRequest(text, model = "text-embedding-ada-002"))
    }.body<EmbeddingResponse>()

    response.data.first().embedding
}


Create the products vector store

In [9]:
@Serializable
data class VectorParams(
    val size: Int,
    val distance: String
)

@Serializable
data class CollectionConfig(
    val vectors: Map<String, VectorParams>  // 👈 named vector config
)

fun createProductsCollection() = runBlocking {
    val config = CollectionConfig(
        vectors = mapOf("default" to VectorParams(size = 1536, distance = "Cosine"))
    )

    val result = client.put("http://localhost:6333/collections/products") {
        contentType(ContentType.Application.Json)
        setBody(config)
    }

    println("🧱 Status: ${result.status}")
    println("🧱 Body: ${result.bodyAsText()}")
}

createProductsCollection()

🧱 Status: 409 Conflict
🧱 Body: {"status":{"error":"Wrong input: Collection `products` already exists!"},"time":0.000895582}


Setup QDrant with our product data

In [14]:
@Serializable
data class QdrantPointInsert(
    val id: String,
    val vector: Map<String, List<Float>>,  // must use "default" as key
    val payload: Map<String, JsonElement>? = null
)

@Serializable
data class QdrantUpsertRequest(
    val points: List<QdrantPointInsert>
)

fun upsertProductsToQdrant(products: List<Product>) = runBlocking {
    val points = products.map { product ->
        val text = "${product.name}. ${product.description}. Category: ${product.category}"
        val vector = embed(text)

        QdrantPointInsert(
            id = UUID.randomUUID().toString(),
            vector = mapOf("default" to vector),  // named vector format
            payload = mapOf(
                "product_id" to Json.encodeToJsonElement(product.product_id),
                "name" to Json.encodeToJsonElement(product.name),
                "description" to Json.encodeToJsonElement(product.description),
                "category" to Json.encodeToJsonElement(product.category)
            )
        )
    }

    val body = QdrantUpsertRequest(points)

    val response = client.put("http://localhost:6333/collections/products/points?wait=true") {
        contentType(ContentType.Application.Json)
        setBody(body)
    }

    println("📦 Upsert response: ${response.status}")
    println("📦 Body: ${response.bodyAsText()}")
}

Make Qdrant searchable for our products

In [15]:
@Serializable
data class NamedVector(
    val name: String,
    val vector: List<Float>
)

@Serializable
data class QdrantSearchRequest(
    val vector: NamedVector,
    val limit: Int,
    @SerialName("with_payload") val withPayload: Boolean
)

@Serializable
data class QdrantSearchHit(
    val id: String,
    val score: Float,
    val payload: Map<String, JsonElement>? = null
)

@Serializable
data class QdrantSearchResponse(
    val result: List<QdrantSearchHit> = emptyList()
)

fun findRelevantProducts(email: String): List<String> = runBlocking {
    val vector = embed(email)
    val body = QdrantSearchRequest(
        vector = NamedVector(name = "default", vector = vector),
        limit = 5,
        withPayload = true
    )

    val response = client.post("http://localhost:6333/collections/products/points/search") {
        contentType(ContentType.Application.Json)
        setBody(body)
    }

    val raw = response.bodyAsText()

    try {
        val json = Json { ignoreUnknownKeys = true }
        json.decodeFromString<QdrantSearchResponse>(raw).result
            .filter { it.score >= 0.80 }
            .mapNotNull { it.payload?.get("product_id")?.jsonPrimitive?.content }
    } catch (e: Exception) {
        println("❌ Parsing failed: ${e.message}")
        emptyList()
    }
}


Associate product ids with each order request

In [17]:
val orderRequests = classifiedResults.filter{ it.requestType == "Order Request" }
val messageById = emailDataFrame.rows().associate { row ->
    row["email_id"] as String to row["message"] as String
}

data class OrderWithMatches(val emailId: String, val message: String, val matchedProductIds: List<String>)

val enrichedOrders = classifiedResults
    .map {
        val originalMessage = messageById[it.emailId] ?: ""
        val matched = findRelevantProducts(originalMessage)
        OrderWithMatches(it.emailId, originalMessage, matched)
    }

enrichedOrders

[OrderWithMatches(emailId=E001, message=Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you!, matchedProductIds=[LTH0976, SWL2345]), OrderWithMatches(emailId=E002, message=Good morning, I'm looking to buy the VBT2345 Vibrant Tote bag. My name is Jessica and I love tote bags, they're so convenient for carrying all my stuff. Last summer I bought this really cute straw tote that I used at the beach. Oh, and a few years ago I got this nylon tote as a free gift with purchase that I still use for groceries., matchedProductIds=[VBT2345, QTP5432, LTH5432, CBG9876, CCB6789]), OrderWithMatches(emailId=E003, message=Hello, I need a new bag to carry my laptop and documents for work. My name is David and I'm having a hard time deciding which would be better - the LTH1098 Leather Backpack or the Leather Tote? Does one have more organizational pockets than the other? A

In [38]:
fun productsToString(products: List<Product>) = products.joinToString("\n") {
    "- ${it.product_id}: ${it.name} — ${it.description} (stock: ${it.stock})"
}

fun createPrompt(emailMessage: String, matchedProducts: List<Product>) = """
You are a product assistant. The customer sent this message:

---
$emailMessage
---

Here are the most relevant products:
${productsToString(matchedProducts)}

Instructions:
- Based on the email and the product descriptions, determine which (if any) of these products the customer is trying to order.
- Only include products that are in stock (stock > 0).
- If one or more matching in-stock products are found, set status to "created".
- If none of the products are in stock, set status to "out of stock".
- Respond with a JSON object containing two keys:
  - matched_products: an array of matching product IDs that are in stock and the quantity of the order.
  - status: either "created" or "out of stock".
  - Make sure it is parseable and no markdown is included.
  - Return an empty array if no items match.
Example response:
{ "matched_products": [
    { productId: "P123", quantity: 2}, { productId: "p456", quantity: 1}
   ], "status": "created" }
"""

@Serializable
data class MatchedProduct(
    val productId: String,
    val quantity: Int
)

@Serializable
data class OrderResponse(
    val matched_products: List<MatchedProduct>,
    val status: String
)

data class OrderResponseSheetRow(
    val emailId: String,
    val productId: String,
    val quantity: Int,
    val status: String
)

val orderSheetRows = enrichedOrders.flatMap { order ->
    val matchedProducts = products.filter { it.product_id in order.matchedProductIds }
    val emailMessage = messageById[order.emailId] ?: ""
    val prompt = createPrompt(emailMessage = emailMessage, matchedProducts = matchedProducts)

    val response = runBlocking {
        client.post("https://api.openai.com/v1/chat/completions") {
            headers {
                append(HttpHeaders.Authorization, "Bearer $apiKey")
                append(HttpHeaders.ContentType, ContentType.Application.Json)
            }
            setBody(OpenAiRequest(
                model = "gpt-3.5-turbo",
                messages = listOf(
                    Message("system", "You are a helpful product assistant."),
                    Message("user", prompt)
                ),
                temperature = 0.2
            ))
        }.body<OpenAiResponse>()
    }


    val gptJson = response.choices.firstOrNull()?.message?.content?.trim()

    val orderResponse = try {
        Json.decodeFromString<OrderResponse>(gptJson ?: "{}")
    } catch (e: Exception) {
        println("❌ Could not parse GPT response: ${e.message}")
        OrderResponse(emptyList(), "error")
    }

    val sheetRow = orderResponse.matched_products.map { matched ->
        OrderResponseSheetRow(
            emailId = order.emailId,
            productId = matched.productId,
            quantity = matched.quantity,
            status = orderResponse.status
        )
    }
    sheetRow
}

In [33]:
val orderStatusDataFrame = orderSheetRows.toDataFrame()

orderStatusDataFrame.writeCSV("order-status.csv")

orderStatusDataFrame

emailId,productId,quantity,status
E001,LTH0976,4,created
E002,VBT2345,1,created
E003,LTH1098,1,created
E004,SFT1098,3,created
E005,CSH1098,1,created
E006,CBT8901,1,created
E007,CLF2109,5,created
E007,FZZ1098,2,created
E008,VSC6789,1,created
E009,DHN0987,1,created


Generate response emails for each order request

In [36]:
fun generateResponseEmail(originalEmail: String, orderResults: String): String = runBlocking {
    val prompt = """
        Customer's Original Email:
        $originalEmail

        Order Processing Results:
        $orderResults

        Instructions:
        - If all items were created successfully, write a professional, warm confirmation email listing the items.
        - If some items are out of stock, apologize politely. (e.g., wait for restock).
        - Match the tone of the customer's original writing style as much as possible (formal, casual, etc.).
        - Do not include a subject or any templating information.
        - Close out the email with

        Thank You,

        BrooksDuBois.

        - Respond professionally and in production-ready English.
        - Respond ONLY with the final email body, no extra commentary.
    """.trimIndent()

    try {
        val response = client.post("https://api.openai.com/v1/chat/completions") {
            headers {
                append(HttpHeaders.Authorization, "Bearer $apiKey")
                append(HttpHeaders.ContentType, ContentType.Application.Json)
            }
            setBody(OpenAiRequest(
                model = "gpt-4o",
                messages = listOf(Message("user", prompt)),
                temperature = 0.0
            ))
        }.body<OpenAiResponse>()

        response.choices.firstOrNull()?.message?.content?.trim() ?: "(No response)"
    } catch (e: Exception) {
        println("❌ Failed to generate response email: ${e.message}")
        "(Error generating email)"
    }
}

data class OrderResponseSheetRow(val emailId: String, val orderResponse: String)
val productsById = products.associate { it.product_id to it }
val orderResponses = orderSheetRows.map { orderResponse ->
    val product = productsById[orderResponse.productId]
    val finalEmailBody = generateResponseEmail(
        originalEmail = messageById[orderResponse.emailId] ?: "",
        orderResults = "${orderResponse.status}, ${orderResponse.quantity} ${product?.name}(s) id: ${product?.product_id} "  // or plain list of items + status
    )
    OrderResponseSheetRow(emailId = orderResponse.emailId, orderResponse = finalEmailBody)
}

In [37]:
val orderResponsesDataFrame = orderResponses.toDataFrame()

orderResponsesDataFrame.writeCSV("order-reponses.csv")

orderResponsesDataFrame

emailId,orderResponse
E001,"Hi there, Thank you for reaching out..."
E002,"Good morning Jessica, Thank you for ..."
E003,"Hello David, Thank you for reaching ..."
E004,"Hi, Thank you for reaching out to us..."
E005,"Good day, Thank you for reaching out..."
E006,"Hey Sam, Thanks for reaching out! I ..."
E007,"Hi Liz, Thank you for reaching out t..."
E007,"Hi Liz, Thank you for reaching out t..."
E008,"Hello, Thank you for your order! We ..."
E009,"Hola, Gracias por contactarnos con t..."


### Handle product inquiries

In [44]:
fun processProductInquiry(emailContent: String, matchedProducts: List<Product>): String = runBlocking {
    val prompt = """
        Email: $emailContent

        Here are the most relevant products:
         ${productsToString(matchedProducts)}

        Respond to the email using the provided context.

        Instructions:
        - Match the tone of the customer's original writing style as much as possible (formal, casual, etc.).
        - Respond professionally and in production-ready English.
        - Do not include a subject or any templating information.
        - Close out the email with

        Thank You,

        BrooksDuBois.

        - Respond ONLY with the final email body, no extra commentary.
    """.trimIndent()

    try {
        val response = client.post("https://api.openai.com/v1/chat/completions") {
            headers {
                append(HttpHeaders.Authorization, "Bearer $apiKey")
                append(HttpHeaders.ContentType, ContentType.Application.Json)
            }
            setBody(OpenAiRequest(
                model = "gpt-4o",
                messages = listOf(Message("user", prompt)),
                temperature = 0.2
            ))
        }.body<OpenAiResponse>()

        response.choices.firstOrNull()?.message?.content?.trim() ?: "(No response)"
    } catch (e: Exception) {
        println("❌ Failed to generate inquiry reply: ${e.message}")
        "(Error generating reply)"
    }
}

data class ProductResponseSheetRow(val emailId: String, val response: String)
val inquiryResponses = classifiedResults
    .filter{ it.requestType == "Product Inquiry"}
    .map {
    val originalMessage = messageById[it.emailId] ?: ""
    val matchedProductIds = findRelevantProducts(originalMessage)
    val matchedProducts = matchedProductIds.map{ productsById[it]!! }
    val response = processProductInquiry(emailContent = originalMessage, matchedProducts = matchedProducts)
    ProductResponseSheetRow(emailId = it.emailId, response = response)
}

inquiryResponses


[ProductResponseSheetRow(emailId=E002, response=Good morning Jessica,

Thank you for reaching out and sharing your love for tote bags — they truly are versatile and stylish companions for any occasion. I'm excited to let you know that we do have the VBT2345 Vibrant Tote in stock, with only a few left! It's designed to add a splash of color to your everyday errands and essentials, making it a delightful addition to your collection.

If you're ready to purchase the Vibrant Tote or have any more questions, feel free to let me know. We're here to help make your shopping experience as smooth as possible.

Thank you for choosing us for your tote bag needs. We look forward to assisting you!

Thank You,

BrooksDuBois.), ProductResponseSheetRow(emailId=E003, response=Hi David,

Thank you for reaching out to us! I understand the challenge of choosing the right bag for your needs. Let me provide you with a bit more insight into these two options to help make your decision easier.

The LTH1098 Lea

In [45]:
val inquiryResponsesDataFrame = inquiryResponses.toDataFrame()

inquiryResponsesDataFrame.writeCSV("inquiry-reponses.csv")

inquiryResponsesDataFrame

emailId,response
E002,"Good morning Jessica, Thank you for ..."
E003,"Hi David, Thank you for reaching out..."
E005,"Good day, Thank you for reaching out..."
E006,"Hey Sam, Thanks for reaching out! Th..."
E009,"Hello, Thank you for reaching out wi..."
E011,"Hi there, Thank you for reaching out..."
E012,"Hey there, I hope you're doing grea..."
E013,"Hi Marco, Thank you for reaching out..."
E015,"Good morning, Thank you for reaching..."
E016,"Hi Claire, Thank you for reaching ou..."
